# Prueba de evaluación continua 2
## Parte 3: Precipitaciones en USA
***
- Autor: Juan A. García Cuevas
- Fecha: 16/10/2016
***

# Enunciado

El sitio Web NOOA del gobierno de EEUU proporciona datasets de datos climáticos a través de esta página Web:

http://www.ncdc.noaa.gov/cdo-web/datasets

Entre ellos tenemos los datasets “Quality Controlled Local Climatological Data (QCLCD)” que se describen aquí:

http://www.ncdc.noaa.gov/data-access/land-based-station-data/land-baseddatasets/quality-controlled-local-climatological-data-qclcd

Entre los datos que se encuentran en los datasets QCLCD están las precipitaciones por años y estaciones. Por ejemplo, podemos descargar los datasets de aquí:

http://www.ncdc.noaa.gov/orders/qclcd/

Y encontraremos ficheros con datos de precipitaciones como este:

    Wban Number, YearMonthDay, Time, Hourly Precip
    03013,19960701,0053,0
    03013,19960701,0153,0
    03013,19960701,0253,0
    03013,19960701,0353,0
    03013,19960701,0453,0
    …

Se pide tomar datos de varios años (queda a la elección del estudiante) de este conjunto de
datasets para las precipitaciones y obtener los siguientes resúmenes:


- Día en que ha habido más precipitaciones.
- Año en que ha habido más precipitaciones (obteniendo la media de cada año)

Se pide realizar el análisis en dos versiones:

- Una utilizando DataFrames y los ficheros de texto que se decargan directamente.
- Una segunda con un paso previo en el que se guardan los datos en un fichero HDF5 (que debe contener los metadatos descriptivos necesarios). Queda a la decisión del estudiante cómo organizar los datos en el fichero.

Y se pide comparar:

- El tamaño en disco que ocupan los datos en cada una de las versiones.
- El tiempo comparado de ejecución de los resúmenes anteriores.

Opcional: finalmente, se platea el almacenar en el fichero HDF5 los resúmenes mismos obtenidos y comparar el tiempo de recuperación de esos datos del fichero con el tiempo tardado en calcularlo. 


 # Resolución

In [1]:
# Importamos librerías

from urllib import urlretrieve, urlcleanup
import zipfile
import os.path
import os, sys
from time import time 
import numpy as np
import pandas as pd

In [2]:
# Vamos a trabajar con los datos de los años 2014 al 2015.
# Por ello, iniciamos las siguientes variables:
anioini = 2014
aniofin = 2015
mesini =  1
mesfin = 12

# Lista de fechas que vamos a tratar
listafechas = []
for anio in range(anioini, aniofin + 1):
    for mes in range(mesini, mesfin + 1):
        listafechas.append(str(anio*100 + mes))
print 'Lista de fechas de los archivos a descargar:\n', listafechas

Lista de fechas de los archivos a descargar:
['201401', '201402', '201403', '201404', '201405', '201406', '201407', '201408', '201409', '201410', '201411', '201412', '201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512']


In [3]:
# Función para descargar un archivo de internet
def download(xmlUrl, xmlPath):
    print 'Descargando', xmlUrl
    tiempo_inicial = time()
#    urlretrieve(xmlUrl, xmlPath)
#    urlcleanup()
    tiempo_final = time() 
    tiempo_ejecucion = tiempo_final - tiempo_inicial
    print '... tiempo de descarga:', round(tiempo_ejecucion, 3), 'sg.'

# Función para leer el contenido de un fichero ZIP
def leer_zip(ficherozip):
    fzip = zipfile.ZipFile(ficherozip, 'r')
    #for item in fzip.namelist():
        #print '\nFichero: ', item
        #print ' - Fecha última modificación:', str(fzip.getinfo(item).date_time)
        #print ' - Sistema que lo creó......:', str(fzip.getinfo(item).create_system)
        #print ' - Tamaño comprimido........:', str(fzip.getinfo(item).compress_size)
        #print ' - Tamaño descomprimido.....:', str(fzip.getinfo(item).file_size)
        #print ' - Tipo de compresión.......:', str(fzip.getinfo(item).compress_type)
    return fzip

In [4]:
# Descargamos los ficheros ZIP de http://www.ncdc.noaa.gov
for fecha in listafechas:
    xmlUrl = 'http://www.ncdc.noaa.gov/orders/qclcd/QCLCD' + fecha + '.zip'
    xmlPath = 'QCLCD' + fecha + '.zip'
    download(xmlUrl, xmlPath)

Descargando http://www.ncdc.noaa.gov/orders/qclcd/QCLCD201401.zip
... tiempo de descarga: 0.0 sg.
Descargando http://www.ncdc.noaa.gov/orders/qclcd/QCLCD201402.zip
... tiempo de descarga: 0.0 sg.
Descargando http://www.ncdc.noaa.gov/orders/qclcd/QCLCD201403.zip
... tiempo de descarga: 0.0 sg.
Descargando http://www.ncdc.noaa.gov/orders/qclcd/QCLCD201404.zip
... tiempo de descarga: 0.0 sg.
Descargando http://www.ncdc.noaa.gov/orders/qclcd/QCLCD201405.zip
... tiempo de descarga: 0.0 sg.
Descargando http://www.ncdc.noaa.gov/orders/qclcd/QCLCD201406.zip
... tiempo de descarga: 0.0 sg.
Descargando http://www.ncdc.noaa.gov/orders/qclcd/QCLCD201407.zip
... tiempo de descarga: 0.0 sg.
Descargando http://www.ncdc.noaa.gov/orders/qclcd/QCLCD201408.zip
... tiempo de descarga: 0.0 sg.
Descargando http://www.ncdc.noaa.gov/orders/qclcd/QCLCD201409.zip
... tiempo de descarga: 0.0 sg.
Descargando http://www.ncdc.noaa.gov/orders/qclcd/QCLCD201410.zip
... tiempo de descarga: 0.0 sg.
Descargando http://w

 ## Cargamos los datos en un dataframe

In [5]:
# Creamos un dataframe para agregar los datos
lista = []

# Leemos los ficheros ZIP y añadimos los datos de precipitaciones al dataframe de trabajo
contador = 0
for fecha in listafechas:
    contador = contador + 1
    
    # Leemos el fichero ZIP
    namezip = 'QCLCD' + fecha + '.zip'
    print '\n********************************************************************************'
    print '*** Tratando fichero', contador, 'de', len(listafechas),':', namezip
    fzip = leer_zip(namezip)

    # Leemos el fichero TXT de precipitaciones
    nametxt = fecha + 'precip.txt'
    ftxt = fzip.read(nametxt)
    fzip.close()
    print '*** Extracto del fichero de precipitaciones:\n', ftxt[0:100], '...'

    outfile = open(nametxt, 'w')
    outfile.write(ftxt)
    outfile.close()    

    dfaux = pd.read_csv(nametxt, index_col=None, header=0)
    lista.append(dfaux)

df = pd.concat(lista) 



********************************************************************************
*** Tratando fichero 1 de 24 : QCLCD201401.zip
*** Extracto del fichero de precipitaciones:
Wban,YearMonthDay,Hour,Precipitation,PrecipitationFlag
03011,20140101,01, , 
03011,20140101,02, , 
0 ...

********************************************************************************
*** Tratando fichero 2 de 24 : QCLCD201402.zip
*** Extracto del fichero de precipitaciones:
Wban,YearMonthDay,Hour,Precipitation,PrecipitationFlag
03011,20140201,01, , 
03011,20140201,02, , 
0 ...

********************************************************************************
*** Tratando fichero 3 de 24 : QCLCD201403.zip
*** Extracto del fichero de precipitaciones:
Wban,YearMonthDay,Hour,Precipitation,PrecipitationFlag
03011,20140301,01, , 
03011,20140301,02, , 
0 ...

********************************************************************************
*** Tratando fichero 4 de 24 : QCLCD201404.zip
*** Extracto del fichero de precip

In [6]:
# Mostramos información del dataframe resultante
print df.shape
df.head()

(39083496, 5)


,Wban,YearMonthDay,Hour,Precipitation,PrecipitationFlag
0,3011,20140101,1,,
1,3011,20140101,2,,
2,3011,20140101,3,,
3,3011,20140101,4,,
4,3011,20140101,5,,


In [7]:
# Convertimos la variable Precipitation a tipo numérico 
df['PrecipitationNum'] = pd.to_numeric(df['Precipitation'], errors = 'coerce')
print df.shape
df.head()

(39083496, 6)


,Wban,YearMonthDay,Hour,Precipitation,PrecipitationFlag,PrecipitationNum
0,3011,20140101,1,,,NaN
1,3011,20140101,2,,,NaN
2,3011,20140101,3,,,NaN
3,3011,20140101,4,,,NaN
4,3011,20140101,5,,,NaN


In [8]:
# Eliminamos los registros que no recojen precipitaciones
df = df[np.isfinite(df['PrecipitationNum'])]
print df.shape
df.head()

(1739420, 6)


,Wban,YearMonthDay,Hour,Precipitation,PrecipitationFlag,PrecipitationNum
251,3011,20140111,12,0.03,,0.03
299,3011,20140113,12,0.01,,0.01
1463,3012,20140130,24,0.01,,0.01
1499,3013,20140101,12,0.01,,0.01
1575,3013,20140104,16,0.01,,0.01


In [9]:
# Creamos una variable año para poder agrupar
df['Year'] = df.YearMonthDay.apply(lambda x: str(x)[0:4])
print df.shape
df.head()

(1739420, 7)


,Wban,YearMonthDay,Hour,Precipitation,PrecipitationFlag,PrecipitationNum,Year
251,3011,20140111,12,0.03,,0.03,2014
299,3011,20140113,12,0.01,,0.01,2014
1463,3012,20140130,24,0.01,,0.01,2014
1499,3013,20140101,12,0.01,,0.01,2014
1575,3013,20140104,16,0.01,,0.01,2014


In [10]:
# Eliminamos las variables que no aportan información relevante para el problema
df = df[['YearMonthDay', 'Year', 'PrecipitationNum']]
print df.shape
df.head()

(1739420, 3)


,YearMonthDay,Year,PrecipitationNum
251,20140111,2014,0.03
299,20140113,2014,0.01
1463,20140130,2014,0.01
1499,20140101,2014,0.01
1575,20140104,2014,0.01


## Hacemos los cálculos con el dataframe 

In [11]:
# Tomamos el tiempo al inicio de los cálculos
timeIniDF = time()

# Calculamos el día que ha habido más precipitaciones
print '\n*** Fecha de mayor precipitación:'
dfByDay = df[['YearMonthDay', 'PrecipitationNum']].groupby('YearMonthDay').aggregate(np.sum)
print "Fecha........:", dfByDay.idxmax()
print "Precipitación:", dfByDay.max()

# Calculamos el año en que ha habido más precipitaciones (obteniendo la media de cada año)
print '\n*** Año de mayor precipitación:'
dfByYear = df.groupby(['Year'])
dfByYear['PrecipitationNum'].aggregate(np.mean)
annualAverage = dfByYear['PrecipitationNum'].aggregate(np.mean)
print(annualAverage)

# Tomamos el tiempo al final de los cálculos y obtenemos el tiempo total
timeFinDF = time() 
timeExeDF = timeFinDF - timeIniDF
print '\n*** Tiempo de cálculo:'
print round(timeExeDF, 3), 'sg.'


*** Fecha de mayor precipitación:
Fecha........: PrecipitationNum    20141016
dtype: int64
Precipitación: PrecipitationNum    13010140.27
dtype: float64

*** Año de mayor precipitación:
Year
2014    30.783033
2015    10.200522
Name: PrecipitationNum, dtype: float64

*** Tiempo de cálculo:
0.368 sg.


In [12]:
# Obtenemos el tamaño de los datos en memoria
sizeMemory = df.size
print "Tamaño en memoria:", sizeMemory

Tamaño en memoria: 5218260


 ## Cargamos los datos en una tabla HDF5

In [13]:
# Importamos la librería para trabajar con HDF5 y Pandas
from pandas import HDFStore, DataFrame

Un fichero HDF5 puede contener una estructura de carpetas arbitraria.
En este ejercicio utilizaremos un único fichero para guardar todos los datos.

In [14]:
# Asignamos nombre a la tabla y al fichero HDF5 de trabajo
hdf5tbl = "tabla/hdf5_precipitaciones"
hdf5filename = "hdf5_precipitaciones.h5"

# Eliminamos el fichero de trabajo si estuviera creado
try:
    os.remove(hdf5filename)
except OSError:
    pass

hdf = HDFStore(hdf5filename)

# Guardamos el dataframe en el fichero hdf5
hdf.put(hdf5tbl, df, format='table', data_columns=True)

# Mostramos información de la tabla HDF5
print hdf[hdf5tbl].shape
hdf[hdf5tbl].head()

(1739420, 3)


,YearMonthDay,Year,PrecipitationNum
251,20140111,2014,0.03
299,20140113,2014,0.01
1463,20140130,2014,0.01
1499,20140101,2014,0.01
1575,20140104,2014,0.01


## Hacemos los cálculos con la tabla HDF5

In [15]:
# Tomamos el tiempo al inicio de los cálculos
timeIniHDF5 = time()

# Calculamos el día que ha habido más precipitaciones
print '\n*** Fecha de mayor precipitación:'
dfByDay = hdf[hdf5tbl][['YearMonthDay', 'PrecipitationNum']].groupby('YearMonthDay').aggregate(np.sum)
print "Fecha........:", dfByDay.idxmax()
print "Precipitación:", dfByDay.max()

# Calculamos el año en que ha habido más precipitaciones (obteniendo la media de cada año)
print '\n*** Año de mayor precipitación:'

dfByDay = hdf[hdf5tbl].groupby(['Year'])
dfByDay['PrecipitationNum'].aggregate(np.mean)
print(dfByDay['PrecipitationNum'].aggregate(np.mean))

dfByYear = df.groupby(['Year'])
dfByYear['PrecipitationNum'].aggregate(np.mean)
annualAverage = dfByYear['PrecipitationNum'].aggregate(np.mean)
print(annualAverage)

# Tomamos el tiempo al final de los cálculos y obtenemos el tiempo total
timeFinHDF5 = time() 
timeExeHDF5 = timeFinHDF5 - timeIniHDF5
print '\n*** Tiempo de cálculo:'
print round(timeExeHDF5, 3), 'sg.'


*** Fecha de mayor precipitación:
Fecha........: PrecipitationNum    20141016
dtype: int64
Precipitación: PrecipitationNum    13010140.27
dtype: float64

*** Año de mayor precipitación:
Year
2014    30.783033
2015    10.200522
Name: PrecipitationNum, dtype: float64
Year
2014    30.783033
2015    10.200522
Name: PrecipitationNum, dtype: float64

*** Tiempo de cálculo:
1.675 sg.


In [16]:
# Obtenemos el tamaño de los datos en disco
sizeDisk = hdf.root.tabla.hdf5_precipitaciones.table.size_on_disk
print "Tamaño en disco:", sizeDisk

Tamaño en disco: 48757296


## Comparamos los resultados

In [17]:
print '*** Cálculos con data frame'
print '- Tiempo de ejecución:', round(timeExeDF, 3), 'sg.'
print '- Tamaño en memoria..:', sizeMemory
print ''
print '*** Cálculos con tabla HDF5'
print '- Tiempo de ejecución:', round(timeExeHDF5, 3), 'sg.'
print '- Tamaño en disco....:', sizeDisk


*** Cálculos con data frame
- Tiempo de ejecución: 0.368 sg.
- Tamaño en memoria..: 5218260

*** Cálculos con tabla HDF5
- Tiempo de ejecución: 1.675 sg.
- Tamaño en disco....: 48757296


Como podemos ver, el tiempo de ejecución y el tamaño del conjunto de datos han sido menores con el data frame que con la tabla HDF5.

NOTTA: ¿No debería haber sido al contrario?...

## Guardamos el resumen en una tabla HDF5 y vemos el tiempo que tarda en recuperarse

In [18]:
# Importanos librerías
import tables as tb
from tables import IsDescription,  Int64Col, Float64Col 

In [19]:
# Asignamos nombre a la tabla y al fichero HDF5 de trabajo
hdf5filename = "hdf5_resumen.h5"

# Eliminamos el fichero de trabajo si estuviera creado
try:
    os.remove(hdf5filename)
except OSError:
    pass

# Creamos un fichero HDF5 en modo append
h5file = tb.open_file(hdf5filename, mode="a", Title="Taba resumen")

dt = np.dtype([('Year', int), ('Precipitaciones', float)])
rows = np.array([(2014, annualAverage[0]), (2015, annualAverage[1])], dtype=dt)

# Creación de una tabla con la estructura creada con el constructor dt
h5file.create_table("/", "precipitaciones", dt)

# Append row
h5file.root.precipitaciones.append(rows)

# Recuperamos los datos
timeIniHDF5 = time()
print ("Media Anual 2014")
print h5file.root.precipitaciones[0]
print ("Media Anual 2015")
print h5file.root.precipitaciones[1]
timeFinHDF5 = time()
timeExeHDF5 = timeFinHDF5 - timeIniHDF5
print '\n*** Tiempo de cálculo:'
print round(timeExeHDF5, 3), 'sg.'

Media Anual 2014
(2014, 30.783032757442573)
Media Anual 2015
(2015, 10.200521592665215)

*** Tiempo de cálculo:
0.001 sg.


In [20]:
for node in h5file:
    print(node)

/ (RootGroup) ''
/precipitaciones (Table(2,)) ''


In [21]:
# Cerramos el fichero HDF5
h5file.close

<bound method File.close of File(filename=hdf5_resumen.h5, title='', mode='a', root_uep='/', filters=Filters(complevel=0, shuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/precipitaciones (Table(2,)) ''
  description := {
  "Year": Int64Col(shape=(), dflt=0, pos=0),
  "Precipitaciones": Float64Col(shape=(), dflt=0.0, pos=1)}
  byteorder := 'little'
  chunkshape := (4096,)
>

# Referencias:

- [Crear y leer ficheros ZIP](http://python-esp.blogspot.com.es/2010/02/crear-y-leer-ficheros-zip.html)
- [Lectura y escritura de ficheros](https://amatellanes.wordpress.com/2013/05/06/lectura-y-escritura-de-ficheros-en-python/)